In [ ]:
"""t = torchaudio.transforms.MelSpectrogram(sample_rate=48000, n_mels=128)
path = "/content/COMMONVOICE"
TSV = "my_test.tsv"

tsvm = os.path.join(path, TSV)
df = pd.read_csv(tsvm, sep="\t")
strings = df['path'].values
folder = os.path.join(path, "clips")

mel_l, wav_l = [], []
i = 0
print(len(strings))
for names in strings:
  filename = os.path.join(folder,names)
  waveform, sample_rate = torchaudio.load(filename)
  mel = t(waveform)
  mel_l.append(mel.shape[2])
  wav_l.append(waveform.shape[1])
  i+=1
  if i%100 == 0:
    print(i)

df["mel_l"] = mel_l
df["wav_l"] = wav_l
new_TSV = "new_" + TSV
tsvm = os.path.join("/content/drive/MyDrive/senior_sound/work", new_TSV)
df.to_csv(tsvm, sep = '\t', index=False)"""

't = torchaudio.transforms.MelSpectrogram(sample_rate=48000, n_mels=128)\npath = "/content/COMMONVOICE"\nTSV = "my_test.tsv"\n\ntsvm = os.path.join(path, TSV)\ndf = pd.read_csv(tsvm, sep="\t")\nstrings = df[\'path\'].values\nfolder = os.path.join(path, "clips")\n\nmel_l, wav_l = [], []\ni = 0\nprint(len(strings))\nfor names in strings:\n  filename = os.path.join(folder,names)\n  waveform, sample_rate = torchaudio.load(filename)\n  mel = t(waveform)\n  mel_l.append(mel.shape[2])\n  wav_l.append(waveform.shape[1])\n  i+=1\n  if i%100 == 0:\n    print(i)\n\ndf["mel_l"] = mel_l\ndf["wav_l"] = wav_l\nnew_TSV = "new_" + TSV\ntsvm = os.path.join("/content/drive/MyDrive/senior_sound/work", new_TSV)\ndf.to_csv(tsvm, sep = \'\t\', index=False)'

In [ ]:
"""from modules import Model_Check
b_model = Model_Check(hyperparams)

text_mask, audio_mask = None, None
text_input = torch.zeros((1, 300)).long()
audio_input = torch.randn((1, 1, 128, 1602)).long()

input = (text_input, text_mask, audio_input, audio_mask)

y = b_model(text_input, text_mask, audio_input, audio_mask)
print(y.shape)
print(summary(b_model, text_input, text_mask, audio_input, audio_mask, show_input=True))
c = 0
for parameter in model.parameters():
    print((parameter.shape))
    if len(parameter.shape) == 1:
      c += parameter.shape[0]
    else:
      c += (parameter.shape[0] * parameter.shape[1])
c"""

'from modules import Model_Check\nb_model = Model_Check(hyperparams)\n\ntext_mask, audio_mask = None, None\ntext_input = torch.zeros((1, 300)).long()\naudio_input = torch.randn((1, 1, 128, 1602)).long()\n\ninput = (text_input, text_mask, audio_input, audio_mask)\n\ny = b_model(text_input, text_mask, audio_input, audio_mask)\nprint(y.shape)\nprint(summary(b_model, text_input, text_mask, audio_input, audio_mask, show_input=True))\nc = 0\nfor parameter in model.parameters():\n    print((parameter.shape))\n    if len(parameter.shape) == 1:\n      c += parameter.shape[0]\n    else:\n      c += (parameter.shape[0] * parameter.shape[1])\nc'

In [ ]:
#!wget -c https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-5.1-2020-06-22/en.tar.gz -P"drive/My Drive/senior_sound/work"
#!tar -xf "drive/My Drive/senior_sound/work/en.tar.gz" -C "drive/My Drive/senior_sound/work/COMMONVOICE"
!mkdir COMMONVOICE
!cp "/content/drive/MyDrive/senior_sound/work/c_en_sound.zip" "/content/COMMONVOICE"

In [ ]:
!cp /content/drive/MyDrive/senior_sound/work/{ph_end_new_my_train.tsv,ph_end_new_my_test.tsv,ph_end_new_my_dev.tsv} /content/COMMONVOICE

In [ ]:
!unzip -n -q "/content/COMMONVOICE/c_en_sound.zip" -d "/content/COMMONVOICE"

In [ ]:
!pip install librosa
!pip install torchaudio #-f https://download.pytorch.org/whl/torch_stable.html
#!pip install https://github.com/PytorchLightning/pytorch-lightning/archive/0.9.0.zip --upgrade
!pip install pytorch-model-summary
!pip install allennlp
!pip install phonemizer
!apt-get install festival espeak-ng mbrola
!pip install pytorch-lightning
!pip install comet-ml
!pip install local-attention
!pip install einops

In [ ]:
from hparams import hyperparams
hyperparams.path_dataset_common

'/content/COMMONVOICE'

In [ ]:
from depen import *
from model import Multi_Synth_pl
from datasets import Common_pl_dataset

from pytorch_model_summary import summary
from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [ ]:
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
            ckpt_path = os.path.join(trainer.checkpoint_callback.dirpath, filename)
            trainer.save_checkpoint(ckpt_path)

In [ ]:
seed_e(42)

In [ ]:
comet_logger = CometLogger(
    save_dir='/content/log/',
    api_key="23CU99n7TeyZdPeegNDlQ5aHf",
    project_name="sound-proj",
    workspace="etzelkut",
    # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
    # experiment_name="default" # Optional
)

CometLogger will be initialized in online mode
[INFO] CometLogger will be initialized in online mode


In [ ]:
dataset_pl = Common_pl_dataset(hyperparams)
dataset_pl.prepare_data()
dataset_pl.setup()
train_loader = dataset_pl.train_dataloader()
steps_per_epoch = int(len(train_loader))
print(steps_per_epoch)

download this version 
 en_1932h_2020-06-22 
  extract files
train set
test set multinumber
test set multinumber
3475


In [ ]:
checkpoint_callback = ModelCheckpoint(
                                      monitor='val_loss',
                                      save_last=True, 
                                      dirpath= "/content/drive/MyDrive/senior_sound/gg/",#os.path.join(path, "/checkpoints"),
                                      filename='samplemodel{epoch}.ckpt',
                                      save_top_k=3,
                                      mode='min',
                                      )
every_epoch = CheckpointEveryNSteps(save_step_frequency = steps_per_epoch, use_modelcheckpoint_filename = True)

In [ ]:
model = Multi_Synth_pl(hyperparams, steps_per_epoch = steps_per_epoch)
#lr_monitor = LearningRateMonitor(logging_interval='step')

performer
glu
performer
glu
performer
glu


In [ ]:
trainer = Trainer(callbacks=[checkpoint_callback, every_epoch], #lr_monitor],
                    logger=comet_logger,
                    gpus=1,
                    profiler=True,
                    #auto_lr_find=True, #set hparams
                    #gradient_clip_val=0.5,
                    check_val_every_n_epoch=5,
                    #early_stop_callback=True,
                    max_epochs = hyperparams.epochs,
                    #min_epochs=400,
                    progress_bar_refresh_rate = 0,
                    deterministic=True,)

GPU available: True, used: True
[INFO] GPU available: True, used: True
TPU available: False, using: 0 TPU cores
[INFO] TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[INFO] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model, dataset_pl)

In [ ]:
checkpoint_name = os.path.join("/content/drive/MyDrive/senior_sound/gg", 'manual_save' + str(228) + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [ ]:
trainer.test()

In [ ]:
#'test_loss': 719.5844116210938,
#test_loss': 724.29150390625
#test_loss': 742.3584594726562